In [3]:
import pyvisa
import time

# Initialize the VISA resource manager
rm = pyvisa.ResourceManager()

# Open the connection to the Keithley 6221 using the correct IP address and connection settings
keithley6221 = rm.open_resource('GPIB0::12::INSTR')

# Verify the connection to the 6221 by querying its ID
print("Keithley 6221 ID: ", keithley6221.query('*IDN?'))

# Define the sweep parameters
current_values = [0.5e-3, 1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3, 10e-3]
on_time = 0.0005  # 500 microseconds
off_time = 0.0005  # 500 microseconds
num_steps = len(current_values)

try:
    # Reset the instrument to default settings
    keithley6221.write('*RST')
    time.sleep(2)

    # Set the source to delta mode
    keithley6221.write('SOUR:DELT:MODE ON')
    
    # Configure the 2182A to communicate with the 6221
    #keithley6221.write('SOUR:DELT:NVPR 1')
    keithley6221.write('SOUR:DELT:ARM')

    # Set the delay between steps (on time and off time)
    keithley6221.write(f'SOUR:DELT:INTL {on_time}')
    keithley6221.write(f'SOUR:DELT:INTD {off_time}')

    # Set the sweep to use the custom list of current values
    keithley6221.write('SOUR:SWE:SPAC LIN')
    keithley6221.write(f'SOUR:SWE:POIN {num_steps}')

    # Load the custom sweep list
    for current in current_values:
        keithley6221.write(f'SOUR:LIST:CURR {current}')

    # Enable output and start the sweep
    keithley6221.write('OUTP ON')
    keithley6221.write('INIT:IMM')

    # Wait for the sweep to complete
    time.sleep(on_time * num_steps + off_time * (num_steps - 1))

    # Turn off the output
    keithley6221.write('OUTP OFF')

except pyvisa.errors.VisaIOError as e:
    print(f"VISA IO Error: {e}")
except Exception as e:
    print(f"Error: {e}")
finally:
    # Close the connection
    keithley6221.close()


Keithley 6221 ID:  KEITHLEY INSTRUMENTS INC.,MODEL 6221,4483467,D04  /700x 

